In [ ]:
%load_ext autoreload
%autoreload 2

import os
import wget
import zipfile
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
# from utils import *
from collections import Counter
from tqdm import tqdm
import time
import geopandas as gpd
import gdown # for downloading files from google drive
import shutil
# ignore warnings
import warnings
import sys
warnings.filterwarnings("ignore")